In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import warnings
import gc
import re
import time
from tqdm import tqdm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [2]:
from sklearn.preprocessing import OrdinalEncoder
from fancyimpute import IterativeImputer
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold
from scipy import stats
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import imblearn

Using TensorFlow backend.


In [3]:
train = pd.read_csv('/Users/saikrishnaboggarapu/Downloads/Insurance/train.csv')
train.shape

(51685, 3)

In [4]:
client = pd.read_csv('/Users/saikrishnaboggarapu/Downloads/Insurance/client_data.csv')
client.shape

(147462, 7)

In [5]:
pay = pd.read_csv('/Users/saikrishnaboggarapu/Downloads/Insurance/payment_history.csv')
pay.shape

(495503, 5)

In [6]:
policy = pd.read_csv('/Users/saikrishnaboggarapu/Downloads/Insurance/policy_data.csv')
policy.shape

(282815, 14)

In [7]:
sub = pd.read_csv('/Users/saikrishnaboggarapu/Downloads/Insurance/sample_sub.csv')
sub.shape

(43707, 2)

In [8]:
train.head()

,Policy ID,Lapse,Lapse Year
0,PID_4928TWH,?,?
1,PID_KBLLEGK,?,?
2,PID_90F0QA3,?,?
3,PID_18F3NHF,?,?
4,PID_SX4QUVO,?,?


In [9]:
client.head()

,Policy ID,NPH_TITLE,NPH_LASTNAME,NPH_SEX,NPH_BIRTHDATE,NAD_ADDRESS1,NAD_ADDRESS2
0,PID_P1UHZRT,Ms.,NaN,F,1997,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
1,PID_GUI5GQJ,Mr.,NPH_LASTNAME_L256UFJ,M,1954,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
2,PID_LECMECN,Mr.,NPH_LASTNAME_3YMQL4X,M,1923,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT
3,PID_BCSLJHG,Mrs.,NPH_LASTNAME_IYMYRSV,F,1973,NaN,NaN
4,PID_YA1G5R6,Ms.,NPH_LASTNAME_HJ3MUUR,F,1935,ADDRESS1_WSRXQ43,ADDRESS2_AIIHWOT


In [10]:
pay.head()

,Policy ID,AMOUNTPAID,DATEPAID,POSTDATE,PREMIUMDUEDATE
0,PID_VTBGKIZ,1287.423813,2018-12-29 00:00:00,2018-12-29 00:00:00,2018-01-11 00:00:00
1,PID_9J9V5YQ,0.000000,2017-04-10 00:00:00,2017-04-10 17:30:00,2017-01-10 00:00:00
2,PID_RT2SI59,3435.950888,2018-12-24 00:00:00,2018-12-28 00:00:00,2019-01-01 00:00:00
3,PID_NCXO0DU,3196.364996,2018-06-26 00:00:00,2018-06-26 00:00:00,2018-01-07 00:00:00
4,PID_FZYCT7V,0.000000,2018-03-12 00:00:00,2018-03-12 17:09:07,2018-01-12 00:00:00


In [11]:
policy.head()

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY
0,PID_EPZDSP8,1/9/2019,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_BPN2LEB,2,222,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB
1,PID_6M6G9IB,1/8/2018,PPR_PRODCD_64QNIHM,2795.069380,NPH_LASTNAME_U2H3GC6,1,111,213380.713197,NLO_TYPE_XTHV3A3,609.054794,AAG_AGCODE_9Z3FBGA,PCL_LOCATCODE_7VFS3EQ,OCCUPATION_IKCIDKW,CATEGORY_R821UZV
2,PID_UL0F7LH,1/8/2017,PPR_PRODCD_KOFUYNN,2492.759107,NPH_LASTNAME_B68RERV,1,111,238857.872515,NLO_TYPE_XAJI0Y6,1339.461987,AAG_AGCODE_Y0LKFF0,PCL_LOCATCODE_SKPRCR4,OCCUPATION_NUJZA7T,CATEGORY_8DALFYO
3,PID_TRGUBTU,1/4/2018,PPR_PRODCD_KOFUYNN,3982.538095,NPH_LASTNAME_NPN3VGI,1,111,74968.903115,NLO_TYPE_XAJI0Y6,7870.961557,AAG_AGCODE_1OCF2N0,PCL_LOCATCODE_SPQHMX5,OCCUPATION_W9XA3KX,CATEGORY_LXSLG6M
4,PID_TODLPIB,1/12/2019,PPR_PRODCD_KOFUYNN,1143.953733,NPH_LASTNAME_9VSNH0E,3,555,238857.872515,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_E31VV8B,PCL_LOCATCODE_0T6GYGX,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB


In [12]:
# Train data has exactly same number of test cases as in submission file
sum(train['Lapse']=='?')

43707

In [13]:
train['Lapse'] = np.where(train['Lapse'] == '?','0',train['Lapse'])
train['Lapse'].value_counts()

0    43707
1     7978
Name: Lapse, dtype: int64

In [14]:
train['Lapse'].unique()

array(['0', '1'], dtype=object)

In [15]:
train['Lapse Year'] = np.where(train['Lapse Year'] == '?',2020,train['Lapse Year'])
train['Lapse Year'].value_counts()

2020    43707
2019     5007
2018     2561
2017      410
Name: Lapse Year, dtype: int64

## Policy

In [16]:
policy['NP2_EFFECTDATE'].dtype

dtype('O')

In [17]:
policy['NP2_EFFECTDATE'] = pd.to_datetime(policy['NP2_EFFECTDATE'],format = "%d/%m/%Y")

In [18]:
policy['NP2_EFFECTDATE'] .head()

0   2019-09-01
1   2018-08-01
2   2017-08-01
3   2018-04-01
4   2019-12-01
Name: NP2_EFFECTDATE, dtype: datetime64[ns]

In [19]:
policy.isnull().mean()*100

Policy ID           0.000000
NP2_EFFECTDATE      0.000000
PPR_PRODCD          0.000000
NPR_PREMIUM         0.019094
NPH_LASTNAME        0.000000
CLF_LIFECD          0.000000
NSP_SUBPROPOSAL     0.000000
NPR_SUMASSURED     33.351131
NLO_TYPE            0.000000
NLO_AMOUNT         68.310380
AAG_AGCODE          0.000000
PCL_LOCATCODE       0.000000
OCCUPATION          0.000000
CATEGORY            0.000000
dtype: float64

In [20]:
policy.shape

(282815, 14)

In [21]:
policy_final = policy.drop_duplicates()
policy_final.shape

(278988, 14)

## Merging raw train and policy data



In [22]:
train.shape, policy.shape

((51685, 3), (282815, 14))

In [23]:
train_policy = pd.merge(policy, train, on='Policy ID',how='inner')
train_policy.shape

(282815, 16)

In [24]:
train_policy_encode = train_policy.copy(deep=True)
train_policy_encode.shape

(282815, 16)

In [25]:
train_policy_encode.head()

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,Lapse,Lapse Year
0,PID_EPZDSP8,2019-09-01,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_BPN2LEB,2,222,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020
1,PID_EPZDSP8,2019-09-01,PPR_PRODCD_KOFUYNN,1310.940330,NPH_LASTNAME_AIAW4OT,1,111,145010.003453,NLO_TYPE_XAJI0Y6,1339.461987,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020
2,PID_EPZDSP8,2019-09-01,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_AIAW4OT,6,555,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020
3,PID_EPZDSP8,2019-09-01,PPR_PRODCD_KOFUYNN,1399.620586,NPH_LASTNAME_ZNVP8GM,6,333,74968.903115,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020
4,PID_EPZDSP8,2019-09-01,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_ZNVP8GM,6,333,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020


In [26]:
train_policy['NP2_EFFECTDATE'].dtype

dtype('<M8[ns]')

In [27]:
train_policy['effectdate_month'] = train_policy['NP2_EFFECTDATE'].dt.month
train_policy['effectdate_month'].head()

0    9
1    9
2    9
3    9
4    9
Name: effectdate_month, dtype: int64

In [28]:
train_policy['effectdate_month'].unique()

array([ 9,  8,  5,  4, 12,  2, 10,  7,  6,  3,  1, 11])

In [29]:
# getting columns based on type
object_cols = train_policy_encode.select_dtypes(include=['object']).columns.to_list()
object_cols.remove('Lapse Year')
object_cols.remove('Policy ID')
object_cols.remove('Lapse')
object_cols.append('effectdate_month')
object_cols.append('CLF_LIFECD')
object_cols.append('NSP_SUBPROPOSAL')
object_cols

['PPR_PRODCD',
 'NPH_LASTNAME',
 'NLO_TYPE',
 'AAG_AGCODE',
 'PCL_LOCATCODE',
 'OCCUPATION',
 'CATEGORY',
 'effectdate_month',
 'CLF_LIFECD',
 'NSP_SUBPROPOSAL']

In [30]:
for col in object_cols:
    if col != 'effectdate_month':
        print(col, policy[col].nunique())

PPR_PRODCD 17
NPH_LASTNAME 25275
NLO_TYPE 6
AAG_AGCODE 591
PCL_LOCATCODE 15
OCCUPATION 240
CATEGORY 6
CLF_LIFECD 6
NSP_SUBPROPOSAL 171


In [31]:
train_policy['Lapse'].dtype

dtype('O')

In [32]:
train_policy['Lapse'] = train_policy['Lapse'].astype('int64')
train_policy['Lapse'].dtype

dtype('int64')

In [33]:
def calc_smooth_mean(df, by, on, m):
    mean = df[on].mean()
    agg = df.groupby(by)[on].agg(['count','mean'])
    counts = agg['count']
    means = agg['mean']
    smooth = (counts*means+m*mean)/(counts+m)
    return smooth

In [34]:
for col in object_cols:
    smooth = calc_smooth_mean(train_policy,col,'Lapse',1000)
    train_policy_encode[col+'target_encode'] = train_policy[col].map(smooth*100)

In [35]:
train_policy_encode.shape

(282815, 26)

In [36]:
train_policy_encode.head()

,Policy ID,NP2_EFFECTDATE,PPR_PRODCD,NPR_PREMIUM,NPH_LASTNAME,CLF_LIFECD,NSP_SUBPROPOSAL,NPR_SUMASSURED,NLO_TYPE,NLO_AMOUNT,AAG_AGCODE,PCL_LOCATCODE,OCCUPATION,CATEGORY,Lapse,Lapse Year,PPR_PRODCDtarget_encode,NPH_LASTNAMEtarget_encode,NLO_TYPEtarget_encode,AAG_AGCODEtarget_encode,PCL_LOCATCODEtarget_encode,OCCUPATIONtarget_encode,CATEGORYtarget_encode,effectdate_monthtarget_encode,CLF_LIFECDtarget_encode,NSP_SUBPROPOSALtarget_encode
0,PID_EPZDSP8,2019-09-01,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_BPN2LEB,2,222,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020,23.568306,20.264413,23.567631,15.631698,8.045713,22.075104,20.862971,16.063972,24.073893,23.700876
1,PID_EPZDSP8,2019-09-01,PPR_PRODCD_KOFUYNN,1310.940330,NPH_LASTNAME_AIAW4OT,1,111,145010.003453,NLO_TYPE_XAJI0Y6,1339.461987,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020,23.728752,19.309456,17.022227,15.631698,8.045713,22.075104,20.862971,16.063972,14.309931,14.306398
2,PID_EPZDSP8,2019-09-01,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_AIAW4OT,6,555,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020,23.568306,19.309456,23.567631,15.631698,8.045713,22.075104,20.862971,16.063972,20.583945,23.591565
3,PID_EPZDSP8,2019-09-01,PPR_PRODCD_KOFUYNN,1399.620586,NPH_LASTNAME_ZNVP8GM,6,333,74968.903115,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020,23.728752,21.510824,23.567631,15.631698,8.045713,22.075104,20.862971,16.063972,20.583945,23.751088
4,PID_EPZDSP8,2019-09-01,PPR_PRODCD_B2KVCE7,265.724174,NPH_LASTNAME_ZNVP8GM,6,333,NaN,NLO_TYPE_DPBHSAH,NaN,AAG_AGCODE_APWOOPE,PCL_LOCATCODE_7SHK7I9,OCCUPATION_NNHJ7XV,CATEGORY_GWW4FYB,0,2020,23.568306,21.510824,23.567631,15.631698,8.045713,22.075104,20.862971,16.063972,20.583945,23.751088


In [37]:
policy_final_agg_encode = train_policy_encode.groupby('Policy ID').agg({'NPR_PREMIUM':['sum','mean','max','min'],
                                                          'NPH_LASTNAME':'nunique',
                                                         'CLF_LIFECD':['count','nunique','max','min'],
                                                         'NPR_SUMASSURED':['sum','mean','max','min'],
                                                         'NLO_AMOUNT': ['sum','mean','max','min'],
                                                          'PPR_PRODCD':['count','nunique'],
                                                         'NSP_SUBPROPOSAL':'nunique',
                                                         'NLO_TYPE':'nunique',
                                                         'AAG_AGCODE':'nunique',
                                                       #  'PCL_LOCATCODE':'nunique',
                                                          'OCCUPATION':'nunique',
                                                         'CATEGORY':'nunique',
                                                    #    'effectdate_monthtarget_encode':'mean',  
                                                        'PPR_PRODCDtarget_encode':['mean','max','min'],
                                                    #    'NPH_LASTNAMEtarget_encode':['mean','max','min'],
                                                        'NLO_TYPEtarget_encode':['mean','max','min'],
                                                     #   'AAG_AGCODEtarget_encode':['mean','max','min'],
                                                        'PCL_LOCATCODEtarget_encode':['mean','max','min'],
                                                        'OCCUPATIONtarget_encode':['mean','max','min'],
                                                        'CATEGORYtarget_encode':['mean','max','min'],
                                                        'CLF_LIFECDtarget_encode':['mean','max','min'],
                                                        'NSP_SUBPROPOSALtarget_encode':['mean','max','min'],}).reset_index()

policy_final_agg_encode.columns = ["_".join(x) for x in policy_final_agg_encode.columns.ravel()]

In [38]:
policy_final_agg_encode.head()

,Policy ID_,NPR_PREMIUM_sum,NPR_PREMIUM_mean,NPR_PREMIUM_max,NPR_PREMIUM_min,NPH_LASTNAME_nunique,CLF_LIFECD_count,CLF_LIFECD_nunique,CLF_LIFECD_max,CLF_LIFECD_min,NPR_SUMASSURED_sum,NPR_SUMASSURED_mean,NPR_SUMASSURED_max,NPR_SUMASSURED_min,NLO_AMOUNT_sum,NLO_AMOUNT_mean,NLO_AMOUNT_max,NLO_AMOUNT_min,PPR_PRODCD_count,PPR_PRODCD_nunique,NSP_SUBPROPOSAL_nunique,NLO_TYPE_nunique,AAG_AGCODE_nunique,OCCUPATION_nunique,CATEGORY_nunique,PPR_PRODCDtarget_encode_mean,PPR_PRODCDtarget_encode_max,PPR_PRODCDtarget_encode_min,NLO_TYPEtarget_encode_mean,NLO_TYPEtarget_encode_max,NLO_TYPEtarget_encode_min,PCL_LOCATCODEtarget_encode_mean,PCL_LOCATCODEtarget_encode_max,PCL_LOCATCODEtarget_encode_min,OCCUPATIONtarget_encode_mean,OCCUPATIONtarget_encode_max,OCCUPATIONtarget_encode_min,CATEGORYtarget_encode_mean,CATEGORYtarget_encode_max,CATEGORYtarget_encode_min,CLF_LIFECDtarget_encode_mean,CLF_LIFECDtarget_encode_max,CLF_LIFECDtarget_encode_min,NSP_SUBPROPOSALtarget_encode_mean,NSP_SUBPROPOSALtarget_encode_max,NSP_SUBPROPOSALtarget_encode_min
0,PID_000OJE4,2680.608650,335.076081,820.226234,191.160414,3,8,4,4,1,182053.717301,45513.429325,45513.429325,45513.429325,1339.461987,1339.461987,1339.461987,1339.461987,8,2,4,2,1,2,1,23.648529,23.728752,23.568306,22.749455,23.567631,17.022227,21.840984,21.840984,21.840984,20.456323,22.075104,15.599981,20.862971,20.862971,20.862971,21.210890,24.865152,14.309931,21.367384,23.751088,14.306398
1,PID_000YE7B,5924.716752,2962.358376,5310.408521,614.308232,1,2,1,1,1,438955.825739,219477.912870,393442.396414,45513.429325,1398.351185,1398.351185,1398.351185,1398.351185,2,2,1,2,1,1,1,21.108639,21.578384,20.638894,20.294929,23.567631,17.022227,21.840984,21.840984,21.840984,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
2,PID_002B54Z,3979.282136,331.606845,566.366602,191.160414,2,12,4,6,1,273080.575952,45513.429325,45513.429325,45513.429325,1339.461987,1339.461987,1339.461987,1339.461987,12,2,6,2,1,2,2,23.648529,23.728752,23.568306,23.022181,23.567631,17.022227,31.724054,31.724054,31.724054,20.944041,22.075104,15.288725,19.064118,20.862971,10.069854,21.547003,24.865152,14.309931,22.013458,23.751088,14.306398
3,PID_0084LXN,4725.586016,337.541858,960.980584,191.160414,2,14,5,5,1,318594.005277,45513.429325,45513.429325,45513.429325,1339.461987,1339.461987,1339.461987,1339.461987,14,2,7,2,1,1,1,23.648529,23.728752,23.568306,23.100102,23.567631,17.022227,21.840984,21.840984,21.840984,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,22.251002,24.865152,14.309931,22.145312,23.751088,14.306398
4,PID_0098YXF,401.191521,401.191521,401.191521,401.191521,1,1,1,1,1,45513.429325,45513.429325,45513.429325,45513.429325,217.975890,217.975890,217.975890,217.975890,1,1,1,1,1,1,1,23.442448,23.442448,23.442448,17.022227,17.022227,17.022227,22.033420,22.033420,22.033420,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398


In [39]:
policy_final_agg_encode=policy_final_agg_encode.rename(columns = {'Policy ID_':'Policy ID'})

In [40]:
policy_final_agg_encode.shape

(51685, 46)

## Merging

In [41]:
train_policy_agg = pd.merge(train, policy_final_agg_encode, on = 'Policy ID',how='left')
train_policy_agg.shape

(51685, 48)

In [42]:
train_policy_agg.head()

,Policy ID,Lapse,Lapse Year,NPR_PREMIUM_sum,NPR_PREMIUM_mean,NPR_PREMIUM_max,NPR_PREMIUM_min,NPH_LASTNAME_nunique,CLF_LIFECD_count,CLF_LIFECD_nunique,CLF_LIFECD_max,CLF_LIFECD_min,NPR_SUMASSURED_sum,NPR_SUMASSURED_mean,NPR_SUMASSURED_max,NPR_SUMASSURED_min,NLO_AMOUNT_sum,NLO_AMOUNT_mean,NLO_AMOUNT_max,NLO_AMOUNT_min,PPR_PRODCD_count,PPR_PRODCD_nunique,NSP_SUBPROPOSAL_nunique,NLO_TYPE_nunique,AAG_AGCODE_nunique,OCCUPATION_nunique,CATEGORY_nunique,PPR_PRODCDtarget_encode_mean,PPR_PRODCDtarget_encode_max,PPR_PRODCDtarget_encode_min,NLO_TYPEtarget_encode_mean,NLO_TYPEtarget_encode_max,NLO_TYPEtarget_encode_min,PCL_LOCATCODEtarget_encode_mean,PCL_LOCATCODEtarget_encode_max,PCL_LOCATCODEtarget_encode_min,OCCUPATIONtarget_encode_mean,OCCUPATIONtarget_encode_max,OCCUPATIONtarget_encode_min,CATEGORYtarget_encode_mean,CATEGORYtarget_encode_max,CATEGORYtarget_encode_min,CLF_LIFECDtarget_encode_mean,CLF_LIFECDtarget_encode_max,CLF_LIFECDtarget_encode_min,NSP_SUBPROPOSALtarget_encode_mean,NSP_SUBPROPOSALtarget_encode_max,NSP_SUBPROPOSALtarget_encode_min
0,PID_4928TWH,0,2020,42911.077278,42911.077278,42911.077278,42911.077278,1,1,1,1,1,340667.737114,340667.737114,340667.737114,340667.737114,16702.717882,16702.717882,16702.717882,16702.717882,1,1,1,1,1,1,1,27.247990,27.247990,27.247990,17.022227,17.022227,17.022227,31.724054,31.724054,31.724054,7.235815,7.235815,7.235815,10.069854,10.069854,10.069854,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
1,PID_KBLLEGK,0,2020,7122.537982,3561.268991,3561.268991,3561.268991,1,2,1,1,1,330107.770053,165053.885026,165053.885026,165053.885026,1376.271841,688.135920,776.012205,600.259636,2,1,1,2,1,1,1,0.519139,0.519139,0.519139,9.017376,17.022227,1.012526,23.235411,23.235411,23.235411,15.867233,15.867233,15.867233,27.855515,27.855515,27.855515,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
2,PID_90F0QA3,0,2020,12329.625672,6164.812836,6164.812836,6164.812836,1,2,1,1,1,504934.051445,252467.025723,252467.025723,252467.025723,2382.425571,1191.212785,1343.332956,1039.092615,2,1,1,2,1,1,1,0.519139,0.519139,0.519139,9.017376,17.022227,1.012526,21.840984,21.840984,21.840984,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
3,PID_18F3NHF,0,2020,2278.189789,2278.189789,2278.189789,2278.189789,1,1,1,1,1,238857.872515,238857.872515,238857.872515,238857.872515,1318.742708,1318.742708,1318.742708,1318.742708,1,1,1,1,1,1,1,12.992411,12.992411,12.992411,17.022227,17.022227,17.022227,22.033420,22.033420,22.033420,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
4,PID_SX4QUVO,0,2020,1619.046308,1619.046308,1619.046308,1619.046308,1,1,1,1,1,145010.003453,145010.003453,145010.003453,145010.003453,332.816358,332.816358,332.816358,332.816358,1,1,1,1,1,1,1,25.638383,25.638383,25.638383,17.022227,17.022227,17.022227,25.195099,25.195099,25.195099,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398


In [43]:
# Check columns with missing data
train_policy_agg.columns[train_policy_agg.isnull().any()]

Index([], dtype='object')

In [44]:
train_policy_agg['Lapse'] = train_policy_agg['Lapse'].astype('int64')
train_policy_agg['Lapse Year'] = train_policy_agg['Lapse Year'].astype('int64')

In [45]:
# Train data
X_train = train_policy_agg.drop(['Policy ID','Lapse','Lapse Year'],axis=1)
y_train = train_policy_agg['Lapse']

In [46]:
y_train.unique()

array([0, 1])

In [47]:
X_train.shape, y_train.shape

((51685, 45), (51685,))

In [48]:
tc_pay_policy_test = train_policy_agg[train_policy_agg['Lapse Year']==2020]
tc_pay_policy_test.shape

(43707, 48)

In [49]:
# Actual test
X_test =  tc_pay_policy_test.drop(['Policy ID','Lapse','Lapse Year'],axis=1)
X_test.shape

(43707, 45)

In [50]:
X_train.head()

,NPR_PREMIUM_sum,NPR_PREMIUM_mean,NPR_PREMIUM_max,NPR_PREMIUM_min,NPH_LASTNAME_nunique,CLF_LIFECD_count,CLF_LIFECD_nunique,CLF_LIFECD_max,CLF_LIFECD_min,NPR_SUMASSURED_sum,NPR_SUMASSURED_mean,NPR_SUMASSURED_max,NPR_SUMASSURED_min,NLO_AMOUNT_sum,NLO_AMOUNT_mean,NLO_AMOUNT_max,NLO_AMOUNT_min,PPR_PRODCD_count,PPR_PRODCD_nunique,NSP_SUBPROPOSAL_nunique,NLO_TYPE_nunique,AAG_AGCODE_nunique,OCCUPATION_nunique,CATEGORY_nunique,PPR_PRODCDtarget_encode_mean,PPR_PRODCDtarget_encode_max,PPR_PRODCDtarget_encode_min,NLO_TYPEtarget_encode_mean,NLO_TYPEtarget_encode_max,NLO_TYPEtarget_encode_min,PCL_LOCATCODEtarget_encode_mean,PCL_LOCATCODEtarget_encode_max,PCL_LOCATCODEtarget_encode_min,OCCUPATIONtarget_encode_mean,OCCUPATIONtarget_encode_max,OCCUPATIONtarget_encode_min,CATEGORYtarget_encode_mean,CATEGORYtarget_encode_max,CATEGORYtarget_encode_min,CLF_LIFECDtarget_encode_mean,CLF_LIFECDtarget_encode_max,CLF_LIFECDtarget_encode_min,NSP_SUBPROPOSALtarget_encode_mean,NSP_SUBPROPOSALtarget_encode_max,NSP_SUBPROPOSALtarget_encode_min
0,42911.077278,42911.077278,42911.077278,42911.077278,1,1,1,1,1,340667.737114,340667.737114,340667.737114,340667.737114,16702.717882,16702.717882,16702.717882,16702.717882,1,1,1,1,1,1,1,27.247990,27.247990,27.247990,17.022227,17.022227,17.022227,31.724054,31.724054,31.724054,7.235815,7.235815,7.235815,10.069854,10.069854,10.069854,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
1,7122.537982,3561.268991,3561.268991,3561.268991,1,2,1,1,1,330107.770053,165053.885026,165053.885026,165053.885026,1376.271841,688.135920,776.012205,600.259636,2,1,1,2,1,1,1,0.519139,0.519139,0.519139,9.017376,17.022227,1.012526,23.235411,23.235411,23.235411,15.867233,15.867233,15.867233,27.855515,27.855515,27.855515,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
2,12329.625672,6164.812836,6164.812836,6164.812836,1,2,1,1,1,504934.051445,252467.025723,252467.025723,252467.025723,2382.425571,1191.212785,1343.332956,1039.092615,2,1,1,2,1,1,1,0.519139,0.519139,0.519139,9.017376,17.022227,1.012526,21.840984,21.840984,21.840984,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
3,2278.189789,2278.189789,2278.189789,2278.189789,1,1,1,1,1,238857.872515,238857.872515,238857.872515,238857.872515,1318.742708,1318.742708,1318.742708,1318.742708,1,1,1,1,1,1,1,12.992411,12.992411,12.992411,17.022227,17.022227,17.022227,22.033420,22.033420,22.033420,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
4,1619.046308,1619.046308,1619.046308,1619.046308,1,1,1,1,1,145010.003453,145010.003453,145010.003453,145010.003453,332.816358,332.816358,332.816358,332.816358,1,1,1,1,1,1,1,25.638383,25.638383,25.638383,17.022227,17.022227,17.022227,25.195099,25.195099,25.195099,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398


In [51]:
y_train[0:5]

0    0
1    0
2    0
3    0
4    0
Name: Lapse, dtype: int64

In [52]:
X_test.head()

,NPR_PREMIUM_sum,NPR_PREMIUM_mean,NPR_PREMIUM_max,NPR_PREMIUM_min,NPH_LASTNAME_nunique,CLF_LIFECD_count,CLF_LIFECD_nunique,CLF_LIFECD_max,CLF_LIFECD_min,NPR_SUMASSURED_sum,NPR_SUMASSURED_mean,NPR_SUMASSURED_max,NPR_SUMASSURED_min,NLO_AMOUNT_sum,NLO_AMOUNT_mean,NLO_AMOUNT_max,NLO_AMOUNT_min,PPR_PRODCD_count,PPR_PRODCD_nunique,NSP_SUBPROPOSAL_nunique,NLO_TYPE_nunique,AAG_AGCODE_nunique,OCCUPATION_nunique,CATEGORY_nunique,PPR_PRODCDtarget_encode_mean,PPR_PRODCDtarget_encode_max,PPR_PRODCDtarget_encode_min,NLO_TYPEtarget_encode_mean,NLO_TYPEtarget_encode_max,NLO_TYPEtarget_encode_min,PCL_LOCATCODEtarget_encode_mean,PCL_LOCATCODEtarget_encode_max,PCL_LOCATCODEtarget_encode_min,OCCUPATIONtarget_encode_mean,OCCUPATIONtarget_encode_max,OCCUPATIONtarget_encode_min,CATEGORYtarget_encode_mean,CATEGORYtarget_encode_max,CATEGORYtarget_encode_min,CLF_LIFECDtarget_encode_mean,CLF_LIFECDtarget_encode_max,CLF_LIFECDtarget_encode_min,NSP_SUBPROPOSALtarget_encode_mean,NSP_SUBPROPOSALtarget_encode_max,NSP_SUBPROPOSALtarget_encode_min
0,42911.077278,42911.077278,42911.077278,42911.077278,1,1,1,1,1,340667.737114,340667.737114,340667.737114,340667.737114,16702.717882,16702.717882,16702.717882,16702.717882,1,1,1,1,1,1,1,27.247990,27.247990,27.247990,17.022227,17.022227,17.022227,31.724054,31.724054,31.724054,7.235815,7.235815,7.235815,10.069854,10.069854,10.069854,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
1,7122.537982,3561.268991,3561.268991,3561.268991,1,2,1,1,1,330107.770053,165053.885026,165053.885026,165053.885026,1376.271841,688.135920,776.012205,600.259636,2,1,1,2,1,1,1,0.519139,0.519139,0.519139,9.017376,17.022227,1.012526,23.235411,23.235411,23.235411,15.867233,15.867233,15.867233,27.855515,27.855515,27.855515,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
2,12329.625672,6164.812836,6164.812836,6164.812836,1,2,1,1,1,504934.051445,252467.025723,252467.025723,252467.025723,2382.425571,1191.212785,1343.332956,1039.092615,2,1,1,2,1,1,1,0.519139,0.519139,0.519139,9.017376,17.022227,1.012526,21.840984,21.840984,21.840984,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
3,2278.189789,2278.189789,2278.189789,2278.189789,1,1,1,1,1,238857.872515,238857.872515,238857.872515,238857.872515,1318.742708,1318.742708,1318.742708,1318.742708,1,1,1,1,1,1,1,12.992411,12.992411,12.992411,17.022227,17.022227,17.022227,22.033420,22.033420,22.033420,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398
4,1619.046308,1619.046308,1619.046308,1619.046308,1,1,1,1,1,145010.003453,145010.003453,145010.003453,145010.003453,332.816358,332.816358,332.816358,332.816358,1,1,1,1,1,1,1,25.638383,25.638383,25.638383,17.022227,17.022227,17.022227,25.195099,25.195099,25.195099,22.075104,22.075104,22.075104,20.862971,20.862971,20.862971,14.309931,14.309931,14.309931,14.306398,14.306398,14.306398


## CatBoost 

In [54]:
random_seed = 12345

preds_cb = []
cb_logloss_score =[]

cbfold=StratifiedKFold(n_splits=10, random_state=random_seed, shuffle=True)
i = 1
print(f"Seed: {random_seed}")
for train_idx, val_idx in cbfold.split(X_train, y_train):
    X_train_2, X_val_2 = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_2, y_val_2 = y_train[train_idx], y_train[val_idx]

    # training the model on oversampled 5 folds of training set
    cb = CatBoostClassifier(n_estimators=3000,
                             verbose = 250,
                             eval_metric='Logloss',
                             depth = 6,
                             learning_rate = 0.01,
                             od_type = "Iter",
                             od_wait = 100,
                             rsm=0.8,
                             random_state=random_seed
                            )

    cb.fit(X_train_2, y_train_2, eval_set=[(X_val_2, y_val_2)])
    # testing on 1 fold of validation set
    validation_preds_cb = cb.predict_proba(X_val_2)

    fea_imp = pd.DataFrame({'col': X_train_2.columns, 'imp': cb.feature_importances_})
    fea_imp = fea_imp.sort_values('imp', ascending=False)
    print(fea_imp.head(10))


    test_preds_cb = cb.predict_proba(X_test)
    preds_cb.append(test_preds_cb)
    
    log_loss_error = round(log_loss(y_val_2, validation_preds_cb), 5)

    print(f"\nFold {i}, log loss Score: {log_loss_error}")
        
    cb_logloss_score.append(log_loss_error)


    i = i+1
    
    print('\nCross validated logloss: {}\n'.format(np.mean(cb_logloss_score)))
        
    print(f'logloss Scores: {cb_logloss_score}\n')

cb_logloss_score

Seed: 12345
0:	learn: 0.6805163	test: 0.6805814	best: 0.6805814 (0)	total: 69.7ms	remaining: 3m 28s
250:	learn: 0.3271963	test: 0.3327482	best: 0.3327482 (250)	total: 3.71s	remaining: 40.7s
500:	learn: 0.3200116	test: 0.3279136	best: 0.3279136 (500)	total: 6.62s	remaining: 33s
750:	learn: 0.3157689	test: 0.3258913	best: 0.3258913 (750)	total: 9.75s	remaining: 29.2s
1000:	learn: 0.3126074	test: 0.3247757	best: 0.3247728 (999)	total: 12.7s	remaining: 25.4s
1250:	learn: 0.3091370	test: 0.3241087	best: 0.3240814 (1242)	total: 15.5s	remaining: 21.6s
1500:	learn: 0.3058779	test: 0.3237409	best: 0.3237182 (1475)	total: 18.3s	remaining: 18.3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3236375709
bestIteration = 1648

Shrink model to first 1649 iterations.
                                col        imp
29        NLO_TYPEtarget_encode_min  21.343829
25      PPR_PRODCDtarget_encode_max  10.470645
26      PPR_PRODCDtarget_encode_min   9.418922
27       NLO_TYPEtarget_enco

750:	learn: 0.3159422	test: 0.3259643	best: 0.3259568 (748)	total: 10.4s	remaining: 31s
1000:	learn: 0.3127383	test: 0.3248355	best: 0.3248273 (999)	total: 14.4s	remaining: 28.7s
1250:	learn: 0.3093637	test: 0.3240503	best: 0.3240483 (1243)	total: 18.2s	remaining: 25.5s
1500:	learn: 0.3061020	test: 0.3233699	best: 0.3233699 (1500)	total: 21.7s	remaining: 21.6s
1750:	learn: 0.3030864	test: 0.3230124	best: 0.3230004 (1729)	total: 24.4s	remaining: 17.4s
2000:	learn: 0.3004396	test: 0.3227414	best: 0.3227319 (1972)	total: 27.2s	remaining: 13.6s
2250:	learn: 0.2979142	test: 0.3224577	best: 0.3224566 (2243)	total: 30.8s	remaining: 10.2s
2500:	learn: 0.2955541	test: 0.3222368	best: 0.3222253 (2498)	total: 33.5s	remaining: 6.67s
2750:	learn: 0.2931084	test: 0.3221188	best: 0.3220647 (2660)	total: 36.2s	remaining: 3.27s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.322064652
bestIteration = 2660

Shrink model to first 2661 iterations.
                                col  

[0.32364,
 0.3234,
 0.31995,
 0.31973,
 0.3158,
 0.32206,
 0.31339,
 0.31598,
 0.32031,
 0.31617]

In [55]:
np.unique(validation_preds_cb)

array([6.67894896e-05, 6.96444790e-05, 7.72267016e-05, ...,
       9.99922773e-01, 9.99930356e-01, 9.99933211e-01])

In [56]:
np.mean(cb_logloss_score)

0.3190430000000001

In [57]:
tc_pay_policy_test['Policy ID'].head()

0    PID_4928TWH
1    PID_KBLLEGK
2    PID_90F0QA3
3    PID_18F3NHF
4    PID_SX4QUVO
Name: Policy ID, dtype: object

In [58]:
final_test_predictions_cb = np.mean(preds_cb,0)
final_test_predictions_cb

array([[8.42123080e-01, 1.57876920e-01],
       [9.99241364e-01, 7.58635997e-04],
       [9.98993989e-01, 1.00601126e-03],
       ...,
       [9.99891310e-01, 1.08690465e-04],
       [9.99825650e-01, 1.74349863e-04],
       [9.98258227e-01, 1.74177325e-03]])

In [59]:
final_test_predictions_cb[:,1]

array([1.57876920e-01, 7.58635997e-04, 1.00601126e-03, ...,
       1.08690465e-04, 1.74349863e-04, 1.74177325e-03])

In [60]:
submission_cb = pd.DataFrame({'Policy ID':tc_pay_policy_test['Policy ID'],
                        'Lapse':final_test_predictions_cb[:,1]})

In [61]:
submission_cb.shape

(43707, 2)

In [62]:
submission_cb.head()

,Policy ID,Lapse
0,PID_4928TWH,0.157877
1,PID_KBLLEGK,0.000759
2,PID_90F0QA3,0.001006
3,PID_18F3NHF,0.030170
4,PID_SX4QUVO,0.339308


In [63]:
file_timestamp = datetime.now().strftime('%m%d%Y_%H%M')
submission_cb.to_csv(f'/Users/saikrishnaboggarapu/Downloads/Insurance/Submission_{file_timestamp}.csv', index=False)